<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> Social Media Analytics - Classification </h1>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-14" pubdate>14th Feb 2018</time>
        </div>
    </div>
</div>

In [2]:
# Importing necassary packages 
import string
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.cross_validation import KFold
from nltk.tokenize import sent_tokenize

from __future__ import division
from scipy import sparse , vstack , hstack

from sklearn.svm import SVC
from sklearn.utils import check_random_state, safe_indexing

import nltk.classify.util
from nltk.classify import MaxentClassifier

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Step 1 : Importing social media dataset

##### Step 1a : Data scrapped and tokenized into sentence level
Imported data set consists of posts which have been tokenized to sentence level with a corresponding column having flags- 1 (talking about drug consumption), and 0 (no mention of drug or treatment)

In [3]:
# Reading the sentenced tokenized dataset
posts = pd.read_csv('SMA_collated_tagged_posts.csv')

### Step 2 :  Preprocessing text columns

Textual data has to be processed before using them as features for the model. Various text mining operations are done on the textual data - like cleaning html tags,special characters ,stripping URL tags etc. We are creating word tokens at the end of this process


In [9]:
# Preprocessing the text column

lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and @tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]


def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
        
    # Break the document into sentences
    for sent in sent_tokenize(document):
        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() 

            # If stopword, ignore token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If punctuation, ignore token and continue
            if all(char in string.punctuation for char in token):
                  continue
            
            yield token

#             Lemmatize the token and yield
            #lemma = lemmatize(token, tag)
            #yield lemma

def lemmatize(token, tag):
    tag = {
       'N': wn.NOUN,
       'V': wn.VERB,
       'R': wn.ADV,
       'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    #return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    #tokens_dict = {}
    tokens_list = []
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        # tokens_list
        tokens_list.append(tokens)
# tokens_dict[count] = inverse_transform(tokens) count += 1
    return tokens_list


#### Step 2a : Preprocessing and creating a list of word tokens

In [10]:
# preprocessed data -List of words (non-string)
words= preprocessing(posts['Tokenized.Sentences'])

#output type is a list of words , here words are not in string format 

#### Step 2b : Creating a list of strings
For further operation with tfidf input should be a string of words/phrase. Followong code will create string of phrase which can be appended into a list


In [46]:
# List of strings - in phrases - ready 
List_strings =[] 
for i in words:
    a = " ".join(str(x) for x in i)
    List_strings.append(a) 

In [ ]:
string_words= ' '.join(map(str,words))

## Step 3 : Feature Generation
### Vectorizing
For feature generation we need to create a matrix of vectorized features. For converting a collection of raw documents to a matrix of features , we use TF-IDF vectorizer (“Term Frequency – Inverse Document Frequency"). It assigns scores/weights to each word , which is considered as a data point for the model

<b>Term Frequency: This summarizes how often a given word appears within a document.</b><br>
<b>Inverse Document Frequency: This downscales words that appear a lot across documents.</b><br>
TF-IDF are word frequency scores that try to highlight words that are more interesting .i.e frequent in a document but not across documents.

The TfidfVectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allow you to encode new documents. Alternately, if you already have a learned CountVectorizer, you can use it with a TfidfTransformer to just calculate the inverse document frequencies and start encoding documents.
The dependent variable in the dataset has skewness which has to be eliminated before passing it through the model . 
The appended list of strings is passed through tfidf and creates are sparse csr matrix , which is used as features for the dataset , with values being the tfidf score


In [14]:
# Dependent variable is defined
y = posts["New_flag"]
subsample_size = 1.0

In [47]:
vectorizer = TfidfVectorizer(use_idf= True)
#vectorizer= CountVectorizer()
X_1 = vectorizer.fit_transform(List_strings)
#Vectorized output is in csr matrix format

### Step 4 : Training and testing split
For classification model , the required training and testing dataset can be prepared with tts function . Output for features and and data point is a sparse matrix , where as the dependent variable is numpy array


In [102]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

#Training and testing split - with a split of 70 : 30 
X_train, X_test, y_train, y_test = tts(X_1,y, test_size=0.3, random_state=111)


### Step 5 : Balancing using Synthetic Minority Over-sampling Technique

This technique is followed to avoid overfitting which occurs when exact replicas of minority instances are added to the main dataset. A subset of data is taken from the minority class as an example and then new synthetic similar instances are created. These synthetic instances are then added to the original dataset. The new dataset is used as a sample to train the classification models.

SMOTE is used to create a balanced dataset which cab passed through the best-fit classifier model. The random_state should be equal to the assigned for the selected model


In [105]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=111,ratio = 1.0)
x_res, y_res = sm.fit_sample(X_train,y_train)

C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


### Running Linear SVM model 
Support vector mahcine is a supervised learning method used for classification, regression and outliers detection. It is memory efficient as it uses a subset of training points in the decision function (called support vectors) . It works good for both binary and multi-class classification . It is flexible as we can select custom kernel as per the training data

SVM is independent of dimensionality of feature space. SVM uses overfitting protection , hence it can handle large number of features which are there in text classifiers



In [106]:
# from sklearn.svm import SVC

classifier = SVC(kernel='linear')
model = classifier.fit(x_res ,y_res)

#preprocessed_test = preprocessing(X_test)
#B =[]
#for i in preprocessed_test:
  #  a = " ".join(str(x) for x in i)
# B.append(a) 
    
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.95      0.93      0.94      1329
          1       0.59      0.69      0.63       190

avg / total       0.91      0.90      0.90      1519



### K-Fold Cross Validation 

Cross Validation is a very useful technique for assessing the performance of machine learning models. It helps in knowing how the machine learning model would generalize to an independent data set.

K-fold is also a  model validation technique for assessing how the results of a statistical analysis will generalize to an independent data set. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice . It randomly splits the entire dataset into k-folds. For each k folds in your dataset ,test the model on k – 1 folds of the data set. The accuracy for each model will tell the effectiveness of the selected model and whether it is overfitting or not

In [107]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X_1):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x_res[train_index], x_res[test_index]
    y_train, y_test = y_res[train_index], y_res[test_index]
    classifier = SVC(kernel='linear')
    model = classifier.fit(X_train ,y_train)
    feature_test = X_test
    y_pred = classifier.predict(feature_test)
    print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.98      0.94      0.96      2173
          1       0.70      0.88      0.78       358

avg / total       0.94      0.93      0.93      2531

             precision    recall  f1-score   support

          0       0.98      0.92      0.95      2172
          1       0.66      0.90      0.76       359

avg / total       0.94      0.92      0.93      2531

             precision    recall  f1-score   support

          0       0.68      0.97      0.80       878
          1       0.98      0.76      0.86      1652

avg / total       0.88      0.83      0.84      2530



In [48]:
Final_posts = pd.read_csv('Final_posts.csv')


## Checking post level accuracy
#### Step 1 : Creating training and testing splits

In [71]:
posts_sorted = posts.sort_values('Docs')
train = posts_sorted['Docs']<=1700
test = posts_sorted['Docs']>1700
training_se = posts_sorted[train]
test_se = posts_sorted[test]


#### Step 2 : Preprocessing and tokenizing into a list

In [72]:
words_se_level = preprocessing(training_se['Tokenized.Sentences'])

In [73]:
List_strings_se_level =[] 
for i in words_se_level:
    a = " ".join(str(x) for x in i)
    List_strings_se_level.append(a)


#### Step 3 : TF- IDF vectorizing

In [74]:
# Training data set creation
X_se_level = vectorizer.fit_transform(List_strings_se_level)
y_se_level = training_se['New_flag']

#### Step 4 : Balancing using SMOTE

In [75]:
sm_se = SMOTE(random_state=111,ratio = 1.0)
x_res_train, y_res_train = sm_se.fit_sample(X_se_level,y_se_level)

C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Step 5 : Preprocessing test dataset

In [82]:
#Testing data creating - Preprocessing and Y_test creation
words_se_level_test = preprocessing(test_se['Tokenized.Sentences'])

List_strings_se_level_test =[] 
for i in words_se_level_test:
    a = " ".join(str(x) for x in i)
    List_strings_se_level_test.append(a)

  

In [83]:
X_se_test = vectorizer.transform(List_strings_se_level_test)
y_se_test = test_se['New_flag']  


In [70]:
x_res_train

<10846x7402 sparse matrix of type '<type 'numpy.float64'>'
	with 241517 stored elements in Compressed Sparse Row format>

#### Step 6 : Running Linear SVM model

In [84]:
classifier = SVC(kernel='linear')
model = classifier.fit(x_res_train ,y_res_train)   
#tfs_test = vectorizer.transform(X_test)
feature_test = X_se_test
from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
y_pred = classifier.predict(feature_test)

print(clsr(y_se_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.97      0.93      0.95      1129
          1       0.62      0.80      0.70       167

avg / total       0.92      0.91      0.92      1296



#### Step 6 : Appending predicted tag to test dataset

In [88]:
#appending predicted to test data set
test_se["predictions"] = y_pred

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Step 7 : Creating post level data set from sentence level predicted dataset

In [89]:
tested_post_level = test_se.groupby('unique_key')['predictions'].agg(max)
test_dict = {'unique_key':tested_post_level.index.tolist(),'Predicted':tested_post_level.values}
test_df = pd.DataFrame(test_dict)

#### Step 8 : Merging dataset to calculate post level accuracy

In [95]:
post_level = pd.merge(Final_posts,test_df[['Predicted','unique_key']] ,on='unique_key')

In [109]:
y_test = post_level['TAG']
y_pred = post_level['Predicted']
pred = sum(post_level['Predicted'])
tag = sum(post_level['TAG'])
Precison = pred/tag
print Precison

0.8478260869565217


In [ ]:
#Getting out top n features  having best tfidf score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

#y = len(string_words.split())
#z = int(round(y*1))
vectorizer = TfidfVectorizer(use_idf= True)
#vectorizer= CountVectorizer()
X = vectorizer.fit_transform(A)
#indices = np.argsort(vectorizer)[::-1]
#features = vectoriz.get_feature_names()
#top_n = z
#top_features = [features[i] for i in indices[:top_n]]

type(X)
X

In [ ]:
feature_test_1 = vectorizer.transform(X_test)
y_test= test_post['New_flag']

In [ ]:
test_post["predictions"] = y_pred


In [ ]:
test_post.to_csv("test_post.csv", index = False)

In [ ]:
data_1 = pd.read_csv('posts_level_data.csv')

In [ ]:
words_1= preprocessing(data_1['posts_p'])
B =[]
for i in words_1:
    a = " ".join(str(x) for x in i)
    B.append(a) 

In [ ]:
B =[]
for i in words_1:
    a = " ".join(str(x) for x in i)
    B.append(a)

In [ ]:

X_2 = vectorizer.transform(B)
X_2

feature = X.tocsc()[:,indices_list]

In [ ]:
y_pred = model.predict(X_2)

In [44]:
#transcript = str(open(transcript_file,'r').read().lower().translate(None, string.punctuation))

response = vectorizer.fit_transform(A) #Need to pickle this and load back as it grows.
pickle.dump(vectorizer, open("tf_idf.p","wb"))
feature_names = vectorizer.get_feature_names()

freq_list = []
temp = set()
for col in response.nonzero()[1]:
    freq_list.append(tuple( (feature_names[col].encode('ascii','ignore'), response[0, col] )) )

top_n_words = [x for x in sorted(freq_list)] #Sorting acc to tfidf value
top_n_words = [ x for x in top_n_words if x[0] ] #not in stopwords.words('english')] #Removing stop words
top_n_words = top_n_words[::-1]
top_n_words = top_n_words[:100]
#print "TOP_N_WORDS (TF-IDF): "+str(len(top_n_words))
print top_n_words

[('00', 0.0), ('00', 0.0), ('00', 0.0), ('000', 0.0), ('000', 0.0), ('000', 0.0), ('000', 0.0), ('005', 0.0), ('008', 0.0), ('010', 0.0), ('011', 0.0), ('023', 0.0), ('03', 0.0), ('03', 0.0), ('04', 0.0), ('05', 0.0), ('07', 0.0), ('07', 0.0), ('08', 0.0), ('0854', 0.0), ('088', 0.0), ('09', 0.0), ('09', 0.0), ('09', 0.0), ('09', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0), ('10', 0.0),

In [ ]:
print(clsr(y_test.tolist(), y_pred.tolist()))

In [ ]:

#TempCount_test = TemporalEntitiesCount(X_test)

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

In [ ]:
from sklearn import svm
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7).fit(Y_TRAIN,feature)
